References:
https://www.youtube.com/watch?v=MfFe6FbWNSw
https://www.kaggle.com/gcdatkin/abnormal-heartbeat-classification


In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))



Num GPUs Available:  1


In [3]:
dfs = [pd.read_csv('E:/DeepLearning/Datasets/ecg_dataset/ptbdb_' + x + '.csv') for x in ['normal', 'abnormal']]

We'll rename the column name.As far as I understood pandas. If you invoke .columns you can find all the column names. Now, in the next section we'll first find the number of columns, by len(df.columns). Then we'll take the range of that, so using range(), we'll find kind of a list [0,1,2,3.....,186,187]. But we'll convert that to a real list using list().

In [4]:
for df in dfs:
    df.columns = list(range(len(df.columns))) # I'll have to test weather it works without the list method.

In [5]:
data = pd.concat(dfs, sort = 'False').sample(frac = 1, random_state = 1).reset_index(drop = True)
data = data.rename({187: 'Label'}, axis=1)
#If you want you can add axis = 0 as we are concating vertically. To print,
#type df and hit enter

In [6]:
y = data['Label'].copy()
X = data.drop('Label', axis=1).copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=1)

In [7]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,177,178,179,180,181,182,183,184,185,186
8540,1.000000,0.834705,0.358711,0.163923,0.096708,0.100137,0.137860,0.131001,0.131001,0.147462,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
10734,1.000000,0.783220,0.479365,0.206803,0.010884,0.078912,0.107937,0.113832,0.116553,0.113832,...,0.063492,0.070748,0.094785,0.142404,0.149206,0.155102,0.0,0.0,0.0,0.0
13652,1.000000,0.789198,0.247764,0.000000,0.039662,0.105148,0.115274,0.105654,0.109030,0.108017,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
5014,1.000000,0.783092,0.592754,0.418357,0.347826,0.314976,0.289372,0.275362,0.267150,0.276812,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
11630,0.990495,0.670835,0.302151,0.000000,0.068534,0.074537,0.141571,0.253127,0.309155,0.310155,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,1.000000,0.668866,0.334133,0.069586,0.106179,0.244151,0.262148,0.238152,0.240552,0.246551,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
5192,0.970892,0.406573,0.051643,0.065728,0.184038,0.205634,0.238498,0.227230,0.234742,0.271362,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
12172,1.000000,0.770696,0.600526,0.543364,0.500657,0.496058,0.500657,0.523653,0.519054,0.520368,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
235,1.000000,0.814031,0.646993,0.079065,0.063474,0.122494,0.208241,0.231626,0.263920,0.279510,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


In [8]:
X_train.shape[1]

187

## Model
Firstly we'll be using a simple neural network. A sequential model and add some dense layers to it. 


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [10]:
#inputs = tf.keras.Input(shape=(X_train.shape[1],))

model = Sequential([
    Dense(units = 150, input_shape=(X_train.shape[1],), activation = 'relu'),
    Dense(units = 100, activation = 'relu'),
    Dense(units = 64, activation = 'relu'),
    Dense(units = 32, activation = 'relu'),
    Dense(units = 128, activation = 'relu'),
    Dense(units = 2, activation = 'softmax'),
])


In [11]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [12]:
model.fit(x=X_train, y=y_train, batch_size=10, epochs=30, validation_split = 0.1,verbose=2)

Epoch 1/30


InternalError:  Blas xGEMM launch failed : a.shape=[1,10,187], b.shape=[1,187,150], m=10, n=150, k=187
	 [[node sequential/dense/MatMul (defined at <ipython-input-12-136e15a8fc2d>:1) ]] [Op:__inference_train_function_846]

Function call stack:
train_function


## Test

In [30]:
predictions = model.predict(x=X_test,batch_size=10,verbose=0)

In [33]:
rounded_predictions = np.argmax(predictions, axis=-1)

for i in rounded_predictions:
    print(i)

1
1
1
1
0
0
0
0
1
0
1
1
1
0
1
0
1
1
1
0
0
0
0
1
1
1
1
1
0
1
0
0
1
1
0
1
0
0
0
1
1
1
1
1
0
0
1
1
1
0
1
0
0
1
1
0
1
1
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
0
1
1
1
1
1
0
0
1
1
1
1
1
1
1
0
1
1
1
1
0
0
1
1
0
1
1
1
0
1
1
1
1
0
1
1
0
1
0
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
0
0
1
1
1
1
1
1
0
1
1
0
1
1
1
1
0
1
0
1
0
0
0
0
0
1
1
0
0
0
0
0
0
1
1
1
1
1
0
1
1
0
1
0
0
0
1
0
1
0
1
1
1
1
1
1
1
1
0
0
1
0
1
0
1
1
1
1
1
0
1
1
1
0
1
0
0
1
1
1
1
1
0
1
1
1
0
1
1
1
0
1
1
1
1
1
0
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
1
0
1
0
0
0
0
0
1
1
0
0
1
0
0
1
1
1
1
0
1
1
1
0
1
0
1
0
1
1
1
0
1
1
0
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
1
0
1
1
0
1
1
1
1
1
0
0
0
1
1
0
1
1
1
1
1
0
1
1
0
1
1
0
1
1
1
1
1
1
0
0
1
1
1
0
1
1
1
1
1
0
1
1
0
0
1
1
1
1
0
0
0
1
0
0
1
0
1
1
1
0
1
1
1
1
0
0
1
1
0
1
1
0
1
0
1
1
1
0
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
0
1
0
1
1
1
0
1
1
1
1
0
1
1
1
0
1
0
1
0
1
1
0
1
1
1
0
1
0
0
1
1
1
0
1
0
0
0
1
0
0
0
1
1
1
1
1
0
0
0
0
1
1
1
1
0
1
1
1
1
0
1
1
1
1
0
0
0
1
0
1
0
1
0
1
0
1
0
1
1
0
1
1
0
0
1
1
0
1
1
1
0
1
1
1
1
0
1
0
1
1
